<a href="https://colab.research.google.com/github/farahbakhsh3/DeepAtariLearning/blob/master/new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://www.dropbox.com/s/nxz7aqtmsiusezd/atari.rar
!apt install unrar
!unrar x -r atari.rar


--2018-10-31 16:37:34--  https://www.dropbox.com/s/nxz7aqtmsiusezd/atari.rar
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601a:1::a27d:701
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/nxz7aqtmsiusezd/atari.rar [following]
--2018-10-31 16:37:34--  https://www.dropbox.com/s/raw/nxz7aqtmsiusezd/atari.rar
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd40324f67114a1fa775db9a606.dl.dropboxusercontent.com/cd/0/inline/AUavqqo7LahtYQN4PGnilYe1L23QnV_1yeCTC9XFHokxSlSGofXVZvKrnWSF3WqLdbTqyFcXDlqvV-RagedL07tHGT4cF6XZM9FHGsEQW6BlW0Nt3u_VBGOdrqDnqqED3TWKU9tpP661lbD_KGY-ldz07kRCjb8kElblspO-GCPOUPmz9byE40_x5AmiCs4jcLQ/file [following]
--2018-10-31 16:37:35--  https://ucd40324f67114a1fa775db9a606.dl.dropboxusercontent.com/cd/0/inline/AUavqqo7LahtYQN4PGnilYe1L23QnV_1yeCTC9XFHokxSlSGofXVZvKrnWSF

In [1]:
#!/usr/bin/env python
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, ConvLSTM2D, MaxPool2D
from keras.layers import BatchNormalization
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from prepare import Sample, model_def
import matplotlib.pyplot as plt


def create_model_ConvLSTM2D(INPUT_SHAPE, OUT_SHAPE, dropout_prob=0.5):
    
    model = Sequential()

    model.add(ConvLSTM2D(32, kernel_size=(3, 3), padding='same', 
                            activation='relu', return_sequences=True, strides=(2, 2), 
                            input_shape=INPUT_SHAPE))
#     model.add(ConvLSTM2D(32, kernel_size=(3, 3), padding='same', 
#                             activation='relu', return_sequences=True, strides=(2, 2)))
    # model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_prob))

#     model.add(ConvLSTM2D(64, kernel_size=(3, 3), padding='same', 
#                             activation='relu', return_sequences=True))
    model.add(ConvLSTM2D(64, kernel_size=(3, 3), padding='same', 
                            activation='relu', return_sequences=True, strides=(2, 2)))
    # model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_prob))

#     model.add(ConvLSTM2D(128, kernel_size=(3, 3), padding='same',  
#                             activation='relu', return_sequences=True))
    model.add(ConvLSTM2D(128, kernel_size=(3, 3), padding='same', 
                            activation='relu', strides=(2, 2)))
    # model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_prob))

    model.add(Flatten())

    model.add(Dense(512, activation='relu'))
    model.add(Dropout(dropout_prob))
    model.add(Dense(OUT_SHAPE, activation='softmax'))
    print(model.summary())

    return model


def create_model_Conv2D(INPUT_SHAPE, OUT_SHAPE, dropout_prob=0.5):

    model = Sequential()
    
    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', 
                        activation='relu', input_shape=INPUT_SHAPE))
    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', 
                        activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_prob))

    model.add(Conv2D(64, kernel_size=(3, 3), padding='same', 
                        activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), padding='same', 
                        activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_prob))

    model.add(Conv2D(128, kernel_size=(3, 3), padding='same', 
                        activation='relu'))
    model.add(Conv2D(128, kernel_size=(3, 3), padding='same', 
                        activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_prob))

    model.add(Flatten())
    
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(dropout_prob))
    model.add(Dense(OUT_SHAPE, activation='softmax'))
    
    print(model.summary())
    
    return model

def series_generator(x_train, y_train, batch_size):
    index = _model_def.window_size
    while True:
        x = []
        y = []
        for i in range(batch_size):
            x.append(x_train[index - _model_def.window_size + i:index + i])
            y.append(y_train[index + i])

        x = np.array(x)
        y = np.array(y)
        index += batch_size

        yield x, y

def convert_data_to_series(x_train, y_train, window_size):
    x_ = []
    for i in range(x_train.shape[0] - window_size):
        x_.append(x_train[i:i + window_size])
    x_train = np.array(x_)
    y_train = y_train[:x_train.shape[0]]

    return x_train, y_train


if __name__ == '__main__':

    _model_def = model_def()
    x_train = np.load("X.npy")
    y_train = np.load("Y.npy")
    y_train = to_categorical(y_train, num_classes=_model_def.OUT_SHAPE)

    epochs = 50
    batch_size = 128

    if _model_def.model == 'Conv2D':
        model = create_model_Conv2D(_model_def.INPUT_SHAPE, _model_def.OUT_SHAPE)
    elif _model_def.model == 'ConvLSTM2D':
        # x_train, y_train = convert_data_to_series(x_train, y_train, _model_def.window_size)
        model = create_model_ConvLSTM2D(_model_def.INPUT_SHAPE, _model_def.OUT_SHAPE)

    print('X_Train Shape: ', x_train.shape)
    print('Y_Train Shape: ', y_train.shape)
    print('---------------------------')
    
    if input('Train from zero: <z>  ,  Retrain by load prev weights: <r>  ::   ') == 'r':
        model.load_weights(_model_def.weights_file)
        print('Model weights loaded : ', _model_def.weights_file) 
    else:
        print('Train from zero.')
    print('---------------------------')
        
    chekpoint = ModelCheckpoint(filepath=_model_def.weights_file, 
                                monitor='val_acc', 
                                save_best_only=True, 
                                save_weights_only=True, 
                                verbose=True)
    
    model.compile(loss='categorical_crossentropy', 
                    optimizer='adam', 
                    metrics=['accuracy'])
    
    if _model_def.model == 'Conv2D':
        history = model.fit(x_train, y_train, 
                            batch_size=batch_size, 
                            epochs=epochs, 
                            shuffle=True, 
                            validation_split=0.05, 
                            callbacks=[chekpoint])

    elif _model_def.model == 'ConvLSTM2D':
        history = model.fit_generator(series_generator(x_train, y_train, batch_size), 
                            steps_per_epoch=x_train.shape[0]//batch_size,
                            epochs=epochs, 
                            workers=4,
                            callbacks=[chekpoint])


    # model.save_weights(_model_def.weights_file)
    print('Best weights saved. ', _model_def.weights_file)
    print('---------------------------')


    history_dict = history.history
    loss_value = history_dict['acc']
    val_loss_value = history_dict['val_acc']
    epoches = range(1,len(loss_value) + 1)

    plt.plot(epoches, loss_value, 'bo', label='Traning accuracy')
    plt.plot(epoches, val_loss_value, 'b', label='Validation accuracy')
    plt.title('Training and Validation accuracy')
    plt.xlabel('Epoches')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

Using TensorFlow backend.


1: Boxing
2: RiverRaid
3: Enduro
4: SeaQuest
5: Tennis
6: SpaceInvaders
Please select your game...(1, 2, 3, 4, ...)  ::   2
---------------------------
Game:  Riverraid ( Riverraid-v0 ) Selected.
---------------------------

1: Conv2D
2: ConvLSTM2D
Please select your model...(1, 2)  ::   2
---------------------------
Model:  ConvLSTM2D  Selected.
---------------------------
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_1 (ConvLSTM2D)  (None, None, 53, 40, 32)  40448     
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 53, 40, 32)  0         
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  (None, None, 27, 20, 64)  221440    
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 27, 20, 64)  0         
_____________________________

IndexError: ignored